#UC Berkeley MIDS DATASCI W261-02, Machine Learning at Scale
##Assignement #1  (version 2016-01-14)
## Submitted by Lei Yang ([leiyang@berkeley.edu](mailto:leiyang@berkeley)), 2016-01-19

###*HW1.0.0*: Define big data. Provide an example of a big data problem in your domain of expertise. 


###Answer: 
Big data is

###*HW1.0.1*: In 500 words (English or pseudo code or a combination) describe how to estimate the bias, the variance, the irreduciable error for a test dataset T when using polynomial regression models of degree 1, 2, 3, 4, 5 are considered. How would you select a model?

###Answer:
blabla

###*HW1.1*: Read through the provided control script (pNaiveBayes.sh)

In [1]:
print 'Done!'

Done!


###*HW1.2*: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh, will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.

####Let's define pNaiveBayes.sh script first, we only need to do this once since it is the same throughout HW1

In [126]:
%%writefile pNaiveBayes.sh
## pNaiveBayes.sh
## Author: Jake Ryland Williams
## Usage: pNaiveBayes.sh m wordlist
## Input:
##       m = number of processes (maps), e.g., 4
##       wordlist = a space-separated list of words in quotes, e.g., "the and of"
##
## Instructions: Read this script and its comments closely.
##               Do your best to understand the purpose of each command,
##               and focus on how arguments are supplied to mapper.py/reducer.py,
##               as this will determine how the python scripts take input.
##               When you are comfortable with the unix code below,
##               answer the questions on the LMS for HW1 about the starter code.

## collect user input
m=$1 ## the number of parallel processes (maps) to run
wordlist=$2 ## if set to "*", then all words are used

## a test set data of 100 messages
data="enronemail_1h.txt" 

## the full set of data (33746 messages)
# data="enronemail.txt" 

## 'wc' determines the number of lines in the data
## 'perl -pe' regex strips the piped wc output to a number
linesindata=`wc -l $data | perl -pe 's/^.*?(\d+).*?$/$1/'`

## determine the lines per chunk for the desired number of processes
linesinchunk=`echo "$linesindata/$m+1" | bc`

## split the original file into chunks by line
split -l $linesinchunk $data $data.chunk.

## assign python mappers (mapper.py) to the chunks of data
## and emit their output to temporary files
for datachunk in $data.chunk.*; do
    ## feed word list to the python mapper here and redirect STDOUT to a temporary file on disk
    ####
    ####
    ./mapper.py $datachunk "$wordlist" > $datachunk.counts &
    ####
    ####
done
## wait for the mappers to finish their work
wait

## 'ls' makes a list of the temporary count files
## 'perl -pe' regex replaces line breaks with spaces
countfiles=`\ls $data.chunk.*.counts | perl -pe 's/\n/ /'`
#echo "$countfiles"

## feed the list of countfiles to the python reducer and redirect STDOUT to disk
####
####
./reducer.py $countfiles > $data.output
####
####

## clean up the data chunks and temporary count files
\rm $data.chunk.*

Overwriting pNaiveBayes.sh


####Define mapper.py & reducer.py, and make them executable
- mapper.py counts the single specified word for the chunk, and output an integer
- reducer.py collates counts from all chunks, and output the total count of the single specified word 

In [100]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
countword = sys.argv[2].lower()
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        for word in line.lower().split()[2:]:
            if countword in word:
                count += 1
print count

Overwriting mapper.py


In [101]:
%%writefile reducer.py
#!/usr/bin/python
import sys
sum = 0
for filename in sys.argv[1:]:
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            sum += int(line)
print sum

Overwriting reducer.py


In [86]:
!chmod a+x pNaiveBayes.sh
!chmod a+x mapper.py
!chmod a+x reducer.py

####<span style="color:red">Results: </span>by checking the ouput file, we know there are 10 counts of word 'assistance'.


In [103]:
!./pNaiveBayes.sh 5 "assistance"
## display final output results
!cat enronemail_1h.txt.output

10


###*HW1.3*: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh, will classify the email messages by a single, user-specified word using the multinomial Naive Bayes Formulation. Examine the word “assistance” and report your results.
   

####Define mapper.py:
- obtains count for each word from the chunk, for spam and non-spam email separately, 
- records all counts in a dictionary, 
- outputs the dictionaries (non-spam count, and spam count), (non)spam counts, and keyword.

In [159]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
# let's use two dictionaries to hold the word counts for spam and non-spam
n_count, s_count = {}, {}
nSpam, nNormal = 0, 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
keyword = sys.argv[2].lower()
with open (filename, "r") as myfile:
    for email in myfile.readlines():
        isSpam = email.split('\t')[1] == '1'
        if isSpam:
            nSpam += 1
            for word in email.lower().split()[2:]: # only use subject & content for modeling
                if word not in s_count:
                    s_count[word] = 1
                else:
                    s_count[word] += 1
        else:
            nNormal += 1
            for word in email.lower().split()[2:]: # only use subject & content for modeling
                if word not in n_count:
                    n_count[word] = 1
                else:
                    n_count[word] += 1
print n_count
print s_count
print nNormal
print nSpam
print "'" + keyword + "'"

Overwriting mapper.py


####Define reducer.py:
- collapse wrod counts from all chunks
- estimate NB model parameters: prior and conditional probabilities
- classify messages that contains the keyword
- **Note:** for messages that don't contain the keyword, the decision is solely based on prior probability, which will always give non-spam prediction, thus we skip those messages and only focus on those with the specified keyword
- output results

####Parameter estimation background:
Assuming *positional independence*, and with *add-one Laplace smoothing*, the multinomial NB conditional probability $P(t | c)$ can be estimated as:
$$
\hat{P}(t\mid c)=\frac{T_{ct}+1}{(\sum_{t^\prime \in V}{T_{ct^\prime}})+B},
$$

where $B=|V|$ is the number of terms in the vocabulary $V$ (including all text classes), and $T_{ct}$ is the count of word *t* in class *c*. 

To classify a message, the posterior probability of class $c$ can be calculated as:
$$
c_{map}=\arg\max_{c\in\mathbb C}[\log{\hat{P}(c)}+\sum_{1\leqslant k \leqslant n_d}{\log{\hat{P}(t_k\mid c)}}],
$$
where $\hat{P}(t_k\mid c)$ is estimated above with *positional independence* assumption as $\hat{P}(t\mid c)$.

In [160]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import math
from sets import Set

n_count, s_count = {}, {}
nSpam, nNormal = 0, 0
counts = []

# scan through each output file from the chunks
for filename in sys.argv[1:]:
    # we first read out the 2 count dictionaries
    with open (filename, "r") as myfile:         
        for line in myfile.readlines():
            cmd = 'counts.append(' + line + ')'
            exec cmd
            
    # we then combine word counts, for non-spam and spam messages, respectively
    for word in counts[0]:
        if word not in n_count:
            n_count[word] = counts[0][word]
        else:
            n_count[word] += counts[0][word]
    
    for word in counts[1]:
        if word not in s_count:
            s_count[word] = counts[1][word]
        else:
            s_count[word] += counts[1][word]
            
    # combine spam and non-spam count
    nNormal += int(counts[2])
    nSpam += int(counts[3])
    
    # pass along the keyword for classification
    keyword = counts[4]
    
    # clear counts for next chunk
    counts = []

# we now estimate NB parameters for the specified word, according to the formular above
testfile = 'enronemail_1h.txt'
print 'Classify messages with key word: ' + keyword
B = len(Set(s_count.keys() + n_count.keys()))
tot_n = sum(n_count.values())
tot_s = sum(s_count.values())
p_word_s = 1.0*((s_count[keyword] if keyword in s_count else 0) + 1) / (tot_s + B)
p_word_n = 1.0*((n_count[keyword] if keyword in n_count else 0) + 1) / (tot_n + B)

# finally we classify the messages which contains the specified word
#### prior probability: same for every message, since it's determined by training data ####
p_s = 1.0*nSpam/(nSpam+nNormal)
p_n = 1.0*nNormal/(nSpam+nNormal)

# print model parameters
print '\n============= Model Parameters ============='
print 'P(spam) = %f' %(p_s)
print 'P(non-spam) = %f' %(p_n)
print 'P(%s|spam) = %f' %(keyword, p_word_s)
print 'P(%s|non-spam) = %f' %(keyword, p_word_n)

#### likelihood: dependend on the frequency of specified word ####
print '\n============= Classification Results ============='
print 'isCorrect \t count \t P(spam) \t P(non-spam) \t Result \t message_id'
with open (testfile, "r") as myfile:  
    for line in myfile.readlines():
        msg = line.lower().split()
        words = msg[2:] # only include words in subject and content
        n_word = sum([1 if keyword in word else 0 for word in words])
        # if the message doesn't contain our keyword, skip it;
        if n_word == 0:
            continue
        #### posterior probability ####
        p_s_word = math.log(p_s) + n_word * math.log(p_word_s)
        p_n_word = math.log(p_n) + n_word * math.log(p_word_n)
        isSpam = True if p_s_word > p_n_word else False
        isCorrect = '1' if isSpam == int(msg[1]) else '0'
        # print results
        print isCorrect + '\t'+str(n_word)+'\t'+ str(p_s_word) + '\t' + str(p_n_word) + '\t' + ('spam' if isSpam else 'non-spam') + '\t' + msg[0] 
        

Overwriting reducer.py


####<span style="color:red">Results: </span>run the NB classifier with keyword 'assistance', the output file are displayed below:
- **Model parameters**: 
 - prior 
 - likelihood
- **Classification results**: 
 - correctness
 - keyword count in the message
 - log probability for spam and non-spam 
 - classification result
 - message ID

In [166]:
!./pNaiveBayes.sh 2 "assistance"
!cat enronemail_1h.txt.output

classify messages with key word: assistance

============= Model Parameters =============
P(spam) = 0.440000
P(non-spam) = 0.560000
P(assistance|spam) = 0.000227
P(assistance|non-spam) = 0.000093

============= Classification Results =============
isCorrect 	 count 	 P(spam) 	 P(non-spam) 	 Result 	 message_id
1	1	-9.21279946713	-9.85907840269	spam	0002.2004-08-01.bg
0	1	-9.21279946713	-9.85907840269	spam	0004.1999-12-10.kaminski
0	1	-9.21279946713	-9.85907840269	spam	0005.1999-12-12.kaminski
1	1	-9.21279946713	-9.85907840269	spam	0010.2001-06-28.sa_and_hp
1	1	-9.21279946713	-9.85907840269	spam	0011.2001-06-28.sa_and_hp
1	1	-9.21279946713	-9.85907840269	spam	0013.2004-08-01.bg
1	3	-25.9964372973	-28.4175982176	spam	0018.2001-07-13.sa_and_hp
1	1	-9.21279946713	-9.85907840269	spam	0018.2003-12-18.gp


###*HW1.4*: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh, will classify the email messages by a list of one or more user-specified words. Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results.
   

####Definition of mapper.py remains the same as it still just counts words for both classes

In [ ]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
# let's use two dictionaries to hold the word counts for spam and non-spam
n_count, s_count = {}, {}
nSpam, nNormal = 0, 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
keywords = sys.argv[2].lower()
with open (filename, "r") as myfile:
    for email in myfile.readlines():
        isSpam = email.split('\t')[1] == '1'
        if isSpam:
            nSpam += 1
            for word in email.lower().split()[2:]: # only use subject & content for modeling
                if word not in s_count:
                    s_count[word] = 1
                else:
                    s_count[word] += 1
        else:
            nNormal += 1
            for word in email.lower().split()[2:]: # only use subject & content for modeling
                if word not in n_count:
                    n_count[word] = 1
                else:
                    n_count[word] += 1
print n_count
print s_count
print nNormal
print nSpam
print "'" + keywords + "'"

####Definition of reducer.py is modified to consider multiple keywords, which we use dictionaries to represent

In [173]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import math
from sets import Set

n_count, s_count = {}, {}
nSpam, nNormal = 0, 0
counts = []

# scan through each output file from the chunks
for filename in sys.argv[1:]:
    # we first read out the 2 count dictionaries
    with open (filename, "r") as myfile:         
        for line in myfile.readlines():
            cmd = 'counts.append(' + line + ')'
            exec cmd
            
    # we then combine word counts, for non-spam and spam messages, respectively
    for word in counts[0]:
        if word not in n_count:
            n_count[word] = counts[0][word]
        else:
            n_count[word] += counts[0][word]
    
    for word in counts[1]:
        if word not in s_count:
            s_count[word] = counts[1][word]
        else:
            s_count[word] += counts[1][word]
            
    # combine spam and non-spam count
    nNormal += int(counts[2])
    nSpam += int(counts[3])
    
    # pass along the keyword for classification
    keywords = counts[4].split()
    
    # clear counts for next chunk
    counts = []

testfile = 'enronemail_1h.txt'
print 'Classify messages with keywords: ' + str(keywords)
   
# we now estimate NB parameters for the specified word, according to the formular above
B = len(Set(s_count.keys() + n_count.keys()))
tot_n = sum(n_count.values())
tot_s = sum(s_count.values())

#### prior probability ####
p_s = 1.0*nSpam/(nSpam+nNormal)
p_n = 1.0*nNormal/(nSpam+nNormal)

#### conditional probabilities for words ####
p_word_s, p_word_n = {}, {}
for word in keywords:
    p_word_s[word] = 1.0*((s_count[word] if word in s_count else 0) + 1) / (tot_s + B)
    p_word_n[word] = 1.0*((n_count[word] if word in n_count else 0) + 1) / (tot_n + B)

# finally we classify the messages which contains the specified word
#### print model parameters ####
print '\n============= Model Parameters ============='
print 'P(spam) = %f' %(p_s)
print 'P(non-spam) = %f' %(p_n)
for word in keywords:
    print 'P(%s|spam) = %f' %(word, p_word_s[word])
    print 'P(%s|non-spam) = %f' %(word, p_word_n[word])

#### likelihood: dependend on the frequency of specified word ####
print '\n============= Classification Results ============='
print 'isCorrect \t count \t P(spam) \t P(non-spam) \t Result \t message_id'
with open (testfile, "r") as myfile:  
    for line in myfile.readlines():
        msg = line.lower().split()
        words = msg[2:] # only include words in subject and content
        #### initialize posterior probability ####
        p_s_word = math.log(p_s)
        p_n_word = math.log(p_n)
        
        #### add likelihood for each keyword ####
        n_word = 0
        for key in keywords:
            n_key = sum([1 if key in word else 0 for word in words])
            n_word += n_key
            p_s_word += n_key * math.log(p_word_s[key])
            p_n_word += n_key * math.log(p_word_n[key])
            
        # if the message doesn't contain any keyword, skip it;
        if n_word == 0:
            continue
        isSpam = True if p_s_word > p_n_word else False
        isCorrect = '1' if isSpam == int(msg[1]) else '0'
        # print results
        print isCorrect + '\t'+str(n_word)+'\t'+ str(p_s_word) + '\t' + str(p_n_word) + '\t' + ('spam' if isSpam else 'non-spam') + '\t' + msg[0] 
        

Overwriting reducer.py


In [174]:
!./pNaiveBayes.sh 2 "assistance valium enlargementWithATypo"
!cat enronemail_1h.txt.output

Classify messages with keywords: ['assistance', 'valium', 'enlargementwithatypo']

============= Model Parameters =============
P(spam) = 0.440000
P(non-spam) = 0.560000
P(assistance|spam) = 0.000227
P(assistance|non-spam) = 0.000093
P(valium|spam) = 0.000038
P(valium|non-spam) = 0.000047
P(enlargementwithatypo|spam) = 0.000038
P(enlargementwithatypo|non-spam) = 0.000047

============= Classification Results =============
isCorrect 	 count 	 P(spam) 	 P(non-spam) 	 Result 	 message_id
1	1	-9.21279946713	-9.85907840269	spam	0002.2004-08-01.bg
0	1	-9.21279946713	-9.85907840269	spam	0004.1999-12-10.kaminski
0	1	-9.21279946713	-9.85907840269	spam	0005.1999-12-12.kaminski
0	1	-11.0045589364	-10.5522255833	non-spam	0009.2003-12-18.gp
1	1	-9.21279946713	-9.85907840269	spam	0010.2001-06-28.sa_and_hp
1	1	-9.21279946713	-9.85907840269	spam	0011.2001-06-28.sa_and_hp
1	1	-9.21279946713	-9.85907840269	spam	0013.2004-08-01.bg
0	1	-11.0045589364	-10.5522255833	non-spam	0016.2003-12-19.gp
0	1	-11.0045

###*HW1.5*: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh, will classify the email messages by all words present.

###*HW1.6*: Benchmark your code with the Python SciKit-Learn implementation of multinomial Naive Bayes